# Visualizando a COVID-19 no RN
> Evolução e estágio atual dos municípios

- toc: true 
- badges: true
- hide_binder_badge: true
- comments: true
- categories: [colab]
- image: images/rn.png

> Caso esteja visualizando este post em um dispositivo móvel, por favor utilize seu dispositivo na horizontal.

Este post reúne análises das ocorrências de COVID-19 nos municípios do Rio Grande do Norte. Todos os municípios apresentados nesta análise tiveram casos suspeitos de COVID-19. Os dados utilizados para análise são fornecidos pela [Secretaria de Saúde Pública](http://www.saude.rn.gov.br/Conteudo.asp?TRAN=ITEM&TARG=223456&ACT=&PAGE=&PARM=&LBL=MAT%C9RIA).

A análise inicial se concentra nos municípios de maior população, comparando suas situações com as situações do RN e do Brasil. Esta análise apresenta diferentes métricas, tanto absolutas (suspeitos e confirmados) quanto relativas (suspeitos e confirmados por 100 mil habitantes). Além disso, apresentamos a análise por dia e em dias decorridos desde o primeiro caso.

> Sempre que alternar entre tipos de condição (suspeito ou confirmado), clique duas vezes na área principal do gráfico para que o zoom se ajuste (a área principal do gráfico tem pano de fundo quadriculado).

A análise final apresenta casos suspeitos e confirmados e usa georeferenciamento para visualizar a trajetória do vírus pelo RN. Nesta análise, apresentamos tanto a evolução dos casos suspeitos como dos casos confirmados. Nos mapas, é possível visualizar informações sobre cada localidade passando o mouse por cima do município desejado.

> É possível animar a linha do tempo da evolução dos casos apertando o botão play, ou visualizar uma data específica usando o slider.

Em cada gráfico, várias outras opções de interação estão disponíveis, podendo ser exploradas pelo menu que aparece no canto superior direito de cada um.

In [1]:
#hide
!pip install plotly==4.5.4

     |████████████████████████████████| 7.1MB 5.3MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [0]:
#hide
import requests
import datetime
from urllib.error import HTTPError

import pandas as pd
import numpy as np
import plotly.express as px


# base variables
initial_date = datetime.date(2020,2,28)
days_elapsed = (datetime.date.today() - initial_date).days + 1
base_url = "https://raw.githubusercontent.com/leobezerra/covid19/master/data/rn_covid_19_boletins"

# dicts for data collection
dates = (f"{(initial_date + datetime.timedelta(days=days)).strftime('%m-%d-%Y')}" for days in range(0, days_elapsed))
csvs = {f"{day}": f"{base_url}/{day}.csv" for day in dates}

# collecting data
dfs = []
for day, url in csvs.items():
    try:
        dfs.append(pd.read_csv(url)) 
    except HTTPError:
        pass
df_mun = pd.concat(dfs)

In [0]:
#hide
df_mun = df_mun.drop(["lat", "lon"], axis=1)
df_mun["log_suspeito"] = np.log1p(df_mun["suspeito"])
df_mun.columns = ['localidade', 'data', 'confirmado', 'suspeito', "log_suspeito"]
df_mun = df_mun.query("suspeito > 0 or confirmado > 0").sort_values("data")

In [0]:
#hide
page = 1
base_url = url = "https://brasil.io/api/dataset/covid19/caso/data?format=json&page=" 
df_list = []

while True:
    json_brazil = requests.get(f"{url}{page}").json()
    if "count" not in json_brazil:
        break
    df_list.append(pd.DataFrame(json_brazil["results"]))
    page += 1

df = pd.concat(df_list)

In [0]:
#hide
def reindex_by_date(df, max_day):
    min_day = min(df["date"])
    df = df.set_index("date")
    dates = pd.date_range(min_day, max_day, name="date")
    return df.reindex(dates).ffill()

In [0]:
#hide

# Parsing the data for Brasil
df_states = df.query("place_type == 'state'")
df_states = df_states[["confirmed", "date", "estimated_population_2019", "state"]]

last_day = max(df_states["date"])
df_states["date"] = pd.to_datetime(df_states["date"])
df_states = df_states.groupby("state").apply(lambda df: reindex_by_date(df, last_day))
df_states = df_states.reset_index("date").sort_values("date")

df_states.columns = ["Data", "Confirmado", "População", "Localidade"]
df_states = df_states.drop(["População"], axis=1)

df_brazil = df_states.groupby("Data").sum().reset_index().assign(Localidade="BR")
df_io = pd.DataFrame(df_brazil, columns=["Confirmado", "Data", "Localidade"])

# Parsing the data for RN
localidades = ["Natal", "Parnamirim", "Mossoró"]
df_maiores = df_mun.query(f"localidade in {localidades} and suspeito > 0")

df_rn = df_mun.groupby("data").sum().reset_index().assign(localidade="RN")
df_maiores = pd.concat([df_maiores, df_rn], sort=True).drop("log_suspeito", axis=1)
df_maiores.columns = ["Confirmado", "Data", "Localidade", "Suspeito"]
df_maiores = pd.concat([df_maiores, df_io], sort=True).query("Confirmado > 0")

# Computing relative metrics
df = df.reset_index(drop=True)
df.loc[(df["state"] == 'RN') & df["city"].isna(), 'city'] = "RN"

df_pop = df.query("state == 'RN'").groupby("city").min()
df_pop = df_pop.reset_index()[["city", "estimated_population_2019"]].dropna()

df_relative = pd.merge(df_maiores, df_pop, left_on="Localidade", right_on="city").drop("city", axis=1)
df_relative.columns = ['Confirmado', 'Data', 'Localidade', 'Suspeito',
       'População']

df_relative["Confirmado / cem mil"] = df_relative["Confirmado"] / df_relative["População"] * 100000
df_relative["Suspeito / cem mil"] = df_relative["Suspeito"] / df_relative["População"] * 100000

# Transforming the data for plotting
ts_mun = df_relative.drop("População", axis=1).melt(id_vars=["Localidade", "Data"], var_name="Condição", value_name="Quantidade")

#### Evolução e estágio atual

##### Acumulado de casos por dia

In [7]:
#hide_input
po = px.line(ts_mun, x="Data", y="Quantidade", animation_frame="Condição", color="Localidade")
po.update_layout(
#    title='Evolução dos casos de COVID-19 no Brasil',
    xaxis_title="Dia",
    yaxis_title="Casos",
    updatemenus=[{"visible": False}],
)

##### Acumulado de casos em dias desde o primeiro caso confirmado

In [0]:
#hide

# Computing day zero
df_relative["Data"] = pd.to_datetime(df_relative["Data"])
start_date = df_relative.groupby("Localidade")["Data"].min().reset_index()
start_date.columns = ["Localidade", "Dia0"]

# Adding day zero to data
df_day0 = pd.merge(start_date, df_relative)
days_elapsed = pd.to_datetime(df_day0["Data"]) - pd.to_datetime(df_day0["Dia0"])
df_day0["Dias"] = days_elapsed.astype('timedelta64[D]')

# Transforming the data for plotting
ts_day0 = df_day0.drop(["População", "Data", "Dia0"], axis=1).melt(id_vars=["Localidade", "Dias"], var_name="Condição", value_name="Quantidade")

In [9]:
#hide_input
po = px.line(ts_day0, x="Dias", y="Quantidade", color="Localidade", animation_frame="Condição")
po.update_layout(
#    title='Evolução dos casos de COVID-19 no Brasil',
    xaxis_title="Dias desde o primeiro caso confirmado",
    yaxis_title="Casos confirmados",
    updatemenus=[{"visible": False}],
)

#### Trajetória de dispersão pelo RN

##### Acumulado de casos suspeitos

In [0]:
#hide
rn = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/24/?formato=application/vnd.geo+json&resolucao=5").json()
for location in rn['features']:
    location['id'] = location['properties']['codarea']

locations = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados/24/distritos").json()
location_list = [{
                  "codarea": municipio["municipio"]["id"],
                  "localidade": municipio["municipio"]["nome"]
                  }
                 for municipio in locations]

id_location = pd.DataFrame(location_list)   
id_location.loc[id_location["localidade"] == "Augusto Severo", "localidade"] = "Campo Grande"      

df_ch = pd.merge(df_mun, id_location)
df_confirmed = df_ch.query("confirmado > 0").sort_values("data")
df_ch["data"] = pd.to_datetime(df_ch["data"]).dt.strftime("%d-%m")
df_confirmed["data"] = pd.to_datetime(df_confirmed["data"]).dt.strftime("%d-%m")

In [19]:
#hide_input
fig = px.choropleth(df_ch,
                    geojson=rn, color="log_suspeito", 
                    locations="codarea", hover_name="localidade",
                    hover_data=["suspeito", "confirmado"],
                    animation_frame="data",
                    center={"lat": -5.78362, "lon": -36.914},
                    color_continuous_scale='reds',
                    #width=1000, height=550, 
                    range_color=(0, df_ch["log_suspeito"].max()),
                    labels={'log_suspeito': 'Casos suspeitos (log)',
                            'confirmado': 'Casos confirmados',
                            'suspeito': 'Casos suspeitos',
                            'localidade': 'Município',
                            'codarea': "Código do município (IBGE)",
                            'data': "Data"},
                    #title="Casos suspeitos de COVID-19 no Rio Grande do Norte",
                    )

fig.update_geos(projection={"scale": 70}, 
                fitbounds="locations", 
                visible=False,
                )

fig.layout.coloraxis.colorbar.showticklabels = False
fig.layout.coloraxis.colorbar.title = "Máximo"
fig.show()

##### Casos confirmados

In [20]:
#hide_input
fig = px.choropleth(df_confirmed,
                    geojson=rn, color="confirmado", 
                    locations="codarea", hover_name="localidade",
                    hover_data=["suspeito", "confirmado"],
                    animation_frame="data",
                    center={"lat": -5.78362, "lon": -36.914},
                    color_continuous_scale='reds',
                    #width=1000, height=550, 
                    range_color=(0, df_ch["confirmado"].max()),
                    labels={'log_suspeito': 'Casos suspeitos (log)',
                            'confirmado': 'Casos confirmados',
                            'suspeito': 'Casos suspeitos',
                            'localidade': 'Município',
                            'codarea': "Código do município (IBGE)",
                            'data': "Data"},
                    #title="Casos confirmados de COVID-19 no Rio Grande do Norte",
                    )

fig.update_geos(projection={"scale": 70}, 
                fitbounds="locations", 
                visible=False,
                )
fig.layout.coloraxis.colorbar.showticklabels = False
fig.layout.coloraxis.colorbar.title = "Máximo"

fig.show()